# TT decomposition in torchtt



In [1]:
import torch as tn
try: 
    import torchtt as tntt
except:
    # pip install
    pass

### Decomposition of a full tensor in TT format

We now create a 4d torch.tensor:

In [2]:
tens_full = tn.reshape(tn.arange(32*16*8*10, dtype = tn.float64),[32,16,8,10])

The TT approximation of a given tensor is 
$\mathsf{x}_{i_1i_2...i_d} = \sum\limits_{r_1,...,r_{d-1}=1}^{R_1,...,R_{d-1}} $

In [3]:
tens_tt = tntt.TT(tens_full)

The original tensor can be recovered using the full() method (also check if it equals the original full tensor):

In [4]:
tens_full2 = tens_tt.full()
print(tn.linalg.norm(tens_full-tens_full2))

tensor(9.5860e-09, dtype=torch.float64)


Using the print function, information about the newly created torchtt.TT instance can be displayed:

In [5]:
print(tens_tt)

TT with sizes and ranks:
N = [32, 16, 8, 10]
R = [1, 2, 2, 2, 1]

Device: cpu, dtype: torch.float64
#entries 180 compression 0.00439453125



Slicing operation can be performed on a tensor in TT format. If all the dimensions are indexed with an integer and the multiindices are valid, a torch.tensor with the corresponding value is returned. Slices can be also used, however the returned object in this case is again a torchtt.TT instance.

In [6]:
print(tens_tt[1,2,3,4])
print(tens_tt[1,1:4,2,:])

tensor(1474., dtype=torch.float64)
TT with sizes and ranks:
N = [3, 10]
R = [1, 2, 1]

Device: cpu, dtype: torch.float64
#entries 26 compression 0.8666666666666667



### Tensor operators

### TT rank rounding

In [8]:
t1 = tntt.randn([10,20,30,40],[1,2,2,2,1])
t2 = tntt.randn([10,20,30,40],[1,2,2,2,1])
t3 = tntt.randn([10,20,30,40],[1,2,2,2,1])
t1, t2, t3 = t1/t1.norm(), t2/t2.norm(), t3/t3.norm()
t = t1+1e-3*t2+1e-6*t3
t_full = t.full()
print(t)

TT with sizes and ranks:
N = [10, 20, 30, 40]
R = [1, 6, 6, 6, 1]

Device: cpu, dtype: torch.float64
#entries 2100 compression 0.00875

